In [49]:
import tweepy
import pymongo

In [50]:
#Setting up Mongo db

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Twitter_Data"]
col = db["Tweets"]

In [68]:
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = ''
        consumer_secret = ''
        access_token = ''
        access_token_secret = ''
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
            #data = self.api.rate_limit_status()
            print("Authentication OK")
            
        except:
            print("Error: Authentication Failed")
            
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
    def get_tweets(self, query, count, page, start, end):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        x = col.delete_many({})
        tweets = []
        tweet_dict = {}
 
        try:
            # call twitter api to fetch tweets
            #fetched_tweets = self.api.search(q = query, rpp = count) 
            for tweet in tweepy.Cursor(self.api.search, q=query, until=end, lang="en").items(count):
                #print(tweet.text)
                cleaned_tweet = self.clean_tweet(tweet.text)
                #print(cleaned_tweet)
                tweet_dict['id'] = tweet.id
                tweet_dict['tweet'] = cleaned_tweet
                x = col.update_one(tweet_dict,{'$set':tweet_dict},upsert=True)
                
                
        except tweepy.TweepError as e:
            print("Error : " + str(e))

def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    api.get_tweets(query = "Microsoft", count=100, page = 1, start=datetime.date.today()-datetime.timedelta(days=30), end=datetime.date.today())

    
if __name__ == "__main__":
    # calling main function
    main()

Authentication OK
{'id': 1266879353805172737, 'tweet': 'RT Microsoft lays off journalists to replace them with AI'}


In [69]:
from pprint import pprint

cursor = col.find({})
for document in cursor: 
    pprint(document)
        


{'_id': ObjectId('5ed4360e451dd0d077710449'),
 'id': 1266882046774906880,
 'tweet': 'Microsoft lays off gt 50 journalists replacing them with AI'}
{'_id': ObjectId('5ed4360e451dd0d07771044b'),
 'id': 1266882032686305285,
 'tweet': 'Microsoft demobilizes journalists and uses Android services'}
{'_id': ObjectId('5ed4360e451dd0d07771044d'),
 'id': 1266881996371963905,
 'tweet': 'Luciferase 060606 Maria Abramovic met with Jacob Rothschild SRS 6 '
          'Cannibals of Dead Rothschild'}
{'_id': ObjectId('5ed4360e451dd0d07771044f'),
 'id': 1266881941447442434,
 'tweet': 'RT For all student out there if you have a group assignment please '
          'use google docs You guys may search it on google It is'}
{'_id': ObjectId('5ed4360e451dd0d077710451'),
 'id': 1266881933247754241,
 'tweet': 'RT Microsoft lays off journalists to replace them with AI '
          'vanguardnews'}
{'_id': ObjectId('5ed4360e451dd0d077710453'),
 'id': 1266881926541062146,
 'tweet': 'New Poly Room Solutions for Micro